In [ ]:
import matplotlib.pyplot as plt
import torch
from torch.quasirandom import SobolEngine
import pandas as pd
import numpy as np
from alse.utils import identify_samples_which_satisfy_constraints, normalize, unnormalize
from mpl_toolkits import mplot3d
# from matplotlib import cm
from alse.utils import read_excel
from alse.alse_workflow import alse
import copy
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
bounds = torch.tensor([[200, 300, 1], [3000, 2700, 4]])
tkwargs = {
    "device": torch.device("cpu"),
    "dtype": torch.float,
}
file_path = "../test_data/8_12_data.xlsx"

X, width, pow_cap, wth = read_excel("../test_data/8_12_data.xlsx",["V (mm/min)", "P (W)", "spot size (mm)"], ["width (mm)", "powder_cap", "wth"])

bounds = torch.tensor([[200, 300, 1], [3000, 2700, 4]])
constraints = [("gt", 2.5), ("gt", 50), ("gt", 6)]

algo = alse(X, bounds, output_param, constraints)

algo.initialize_model(["reg", "reg", "reg"])

In [ ]:
N=20
pred, overlap = algo.get_posterior_grid(N)
minus_15, plus_15 = algo.get_test_points(num_points=10, uncertainty=0.15)

x=np.linspace(bounds[0][0], bounds[1][0], N)
y=np.linspace(bounds[0][1], bounds[1][1], N)
z=np.linspace(bounds[0][2], bounds[1][2], N)

Xplt, Yplt, Zplt = np.meshgrid(x,y,z)
Xplt = torch.from_numpy(Xplt)
Yplt = torch.from_numpy(Yplt)
Zplt = torch.from_numpy(Zplt)

fig = go.Figure(
    go.Isosurface(
        x=Xplt.flatten(),
        y=Yplt.flatten(),
        z=Zplt.flatten(),
        value=overlap,
        isomin=0,
        isomax=1,
        surface_count=4,  # number of isosurfaces, 2 by default: only min and max
        colorbar_nticks=4,  # colorbar ticks correspond to isosurface values
        caps=dict(x_show=False, y_show=False),
    )
)
fig.add_trace(
    go.Scatter3d(
        x=minus_15[:,0],
        y=minus_15[:,1],
        z=minus_15[:,2],
        # text=[f"{yname[i]} : {value}" for value in dff[yname[i]]],
        mode="markers",
    )
)
fig.add_trace(
    go.Scatter3d(
        x=plus_15[:,0],
        y=plus_15[:,1],
        z=plus_15[:,2],
        # text=[f"{yname[i]} : {value}" for value in dff[yname[i]]],
        mode="markers",
    )
)
fig.show()

In [ ]:
print(plus_15)

In [ ]:
print(minus_15)